In [1]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from scipy.interpolate import interp2d
from scipy.signal import argrelextrema
from bmcs_shear.api import CrackBridgeAdv
from bmcs_shear.dic_crack import \
    DICGrid, DICStateFields
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

In [2]:
dic_grid = DICGrid(start_t=0, end_t=1, U_factor=100, dir_name='B1_TV1')

In [3]:
dsf = DICStateFields(dic_grid=dic_grid, t=0.5)
dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.01);
dsf.eval()

# Identify the time index corresponding to 75 % of the ultimate load

**Remark:** do it in dic_grid

# Identify the cracks that cross the mid height of the section

In [4]:
x_NC, y_NC, N_tip_C, arg_x_NC = dsf.cracks

In [5]:
n_M = dic_grid.n_x

In [7]:
N_tip_C

array([19, 18, 17, 19, 19, 17, 13,  0])

In [8]:
n_N, n_C = arg_x_NC.shape

In [9]:
mid_N = int(n_N / 3)

In [10]:
primary_cracks = np.where(N_tip_C >= mid_N)[0]
primary_cracks

array([0, 1, 2, 3, 4, 5, 6])

To extract the opening and sliding displacements along the crack path, let us
construct an interploator which takes the input displacements $x_{IJ}$.
Given the line of the crack x(s), y(s) find a normal to the crack from the point $x_{IJ}$

In [30]:
def get_f_ironed(x, y, r=10):
    '''Averaging of a function using a bell-shaped ironing function
    '''
    RR = r
    n = len(x)
    delta_x = x[None, ...] - x[..., None]
    r2_n = (delta_x ** 2) / (2 * RR ** 2)
    alpha_r_MJ = np.exp(-r2_n)
    a_M = np.trapz(alpha_r_MJ, x, axis=-1)
    normed_a_MJ = np.einsum('MJ,M->MJ', alpha_r_MJ, 1 / a_M)
    y_MJ = np.einsum('MJ,J->MJ', normed_a_MJ, y)
    y_smooth = np.trapz(y_MJ, x, axis=-1)
    return y_smooth

In [12]:
from scipy.interpolate import CubicSpline, UnivariateSpline

In [13]:
C = 4
R = 20
x_C, y_C = x_NC[:, C], y_NC[:, C]
x_C_irn = get_f_ironed(y_C, x_C, R)

In [14]:
cs = CubicSpline(y_C[:N_tip_C[C]+3], x_C_irn[:N_tip_C[C]+3], bc_type='clamped')

In [15]:
fig, ax = plt.subplots(1,1)
#ax.plot(x_C_irn[:tip_NC[C]], y_C[:tip_NC[1]], color='orange')
ax.plot(x_C, y_C, 'o')
y_range = np.linspace(y_C[0], y_C[N_tip_C[C]+2], 20)
ax.plot(cs(y_range), y_range, color='green');
ax.axis('equal');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
d_x = 30
X_right = np.array([cs(y_range) + d_x, y_range], dtype=np.float_).T
X_left = np.array([cs(y_range) - d_x, y_range], dtype=np.float_).T
U_C = dsf.interp_U(X_right) - dsf.interp_U(X_left)
U_C

array([[ 2.23104361e-01,  4.87742458e-01],
       [ 2.14781342e-01,  4.68651023e-01],
       [ 2.03563809e-01,  4.63273397e-01],
       [ 1.90493296e-01,  4.69461799e-01],
       [ 1.82462337e-01,  4.67287407e-01],
       [ 1.76399521e-01,  4.63358885e-01],
       [ 1.61187015e-01,  4.56115421e-01],
       [ 1.44598783e-01,  4.51192745e-01],
       [ 1.34447412e-01,  4.55495451e-01],
       [ 1.18839999e-01,  4.57774245e-01],
       [ 1.03639556e-01,  4.61349269e-01],
       [ 8.83807032e-02,  4.52894975e-01],
       [ 7.07824266e-02,  4.37567971e-01],
       [ 5.15279397e-02,  4.40851623e-01],
       [ 2.79464212e-02,  4.50042910e-01],
       [ 2.43120522e-02,  4.33451013e-01],
       [ 2.83191199e-02,  4.18053585e-01],
       [ 2.28072556e-02,  4.04652535e-01],
       [ 1.38142772e-02,  4.03289682e-01],
       [-4.23829430e-04,  4.07010835e-01]])

# Identify the index ranges corresponding to the cracks

In [17]:
xx_MN, yy_MN, cd_field_irn_MN = dsf.crack_detection_field

In [18]:
nn_M, nn_N = xx_MN.shape

In [19]:
arg_x_NP = arg_x_NC[:,primary_cracks]

In [20]:
first_x_NP = np.zeros((mid_N,1), dtype=np.int_)
last_x_NP = np.zeros((mid_N,1), dtype=np.int_) + nn_M
arg_mid_NP = np.hstack([ 
    first_x_NP, arg_x_NP[:mid_N,:], last_x_NP
])
arg_mid_NP

array([[  0,   9,  17,  29,  43,  62,  76,  94, 116],
       [  0,  10,  17,  29,  43,  61,  76,  94, 116],
       [  0,  10,  17,  29,  43,  61,  76,  94, 116],
       [  0,  10,  17,  29,  43,  61,  76,  94, 116],
       [  0,  10,  17,  29,  43,  60,  77,  94, 116],
       [  0,  10,  16,  29,  43,  60,  77,  94, 116],
       [  0,  11,  16,  29,  43,  60,  77,  94, 116],
       [  0,  10,  16,  29,  42,  60,  76,  95, 116],
       [  0,  10,  15,  29,  42,  61,  76,  94, 116]])

In [21]:
# BNT - boundary, row, tooth
teeths_BNT = np.array([
    arg_mid_NP[:, :-1],
    arg_mid_NP[:, 1:]
]
)
teeths_TNB = np.einsum('BNT->TNB', teeths_BNT)

In [22]:
teeths_TNB[0, :, 0] + 3

array([3, 3, 3, 3, 3, 3, 3, 3, 3])

In [23]:
teeth_list_aP = []
for tooth_NB in teeths_TNB[1:]:
    tooth_aP = [
        np.array([np.arange(*tooth_B),
                  np.ones(tooth_B[1]-tooth_B[0])*row],
                 dtype=np.int_)
        for row, tooth_B in enumerate(tooth_NB)
    ]
    teeth_list_aP.append(np.hstack(tooth_aP))

In [24]:
teeth_list_aP[-1]

array([[ 94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
        107, 108, 109, 110, 111, 112, 113, 114, 115,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
        111, 112, 113, 114, 115,  94,  95,  96,  97,  98,  99, 100, 101,
        102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
        115,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
        106, 107, 108, 109, 110, 111, 112, 113, 114, 115,  94,  95,  96,
         97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
        110, 111, 112, 113, 114, 115,  94,  95,  96,  97,  98,  99, 100,
        101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
        114, 115,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,  95,  96,
         97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
        110, 111, 112, 113, 114, 115,  94,  95,  96

In [25]:
xx_MN[teeth_list_aP[2][0], teeth_list_aP[2][1]][::10]

array([324.07827288, 434.22624731, 390.16705754, 346.10786777,
       456.25584219, 412.19665242, 368.13746265, 324.07827288,
       434.22624731, 390.16705754, 346.10786777, 456.25584219,
       423.21144987])

In [26]:
arg_x_NP.shape

(28, 7)

# Get the indexes along the crack ligament

In [27]:
arg_x_NP_left = arg_x_NP[:, :, np.newaxis] + np.arange(-2, 0)[np.newaxis, np.newaxis, :];
arg_x_NP_right = arg_x_NP[:, :, np.newaxis] + np.arange(1, 3)[np.newaxis, np.newaxis, :]

In [28]:
arg_x_NP_left[0,0,:], arg_x_NP_right[0,0,:]

(array([7, 8]), array([10, 11]))

In [29]:
arg_x_NP[0,0]

9

The indexes are obtained on a resolution xx - i.e. the grid used for interpolation of 
the crack detection field.

- the evaluation of sliding and opening makes only sense in the localized range
- construct an `xx` interpolation of the displacement field over the whole domain.
- construct the crack path and smooth it
- construct the ortho-normal bases along the crack path (available in crack-path object)
- discretize along the normal direction within a predefined range
- pick up the global displacements $U_{na}$ along the normal vectors
- use the ortho-normal bases to transform the displacements into the local coordinate system
- center the displacement vector relative to the displacement at the crack position
- find the maximum difference in tangential and normal directions 

# How to define the fixed and rotating frames

Is the current implementation applicable also for the aligned grid analysis?
Actually, it should and it should.